<a href="https://colab.research.google.com/github/gptix/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module3-join-and-reshape-data/LS_DS_113_Join_and_Reshape_Data_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

# Import Data

In [195]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-10-31 00:34:46--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.32.86
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.32.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  97.5MB/s    in 2.0s    

2019-10-31 00:34:48 (97.5 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [196]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [197]:
# MAGIC COMMAND
%cd instacart_2017_05_01

/content/instacart_2017_05_01/instacart_2017_05_01


In [198]:
# ls command options
# -l is for "lots"
# -h is for "human-readable numbers"
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

In [0]:
import pandas as pd


These are the top 10 most frequently ordered products. 
1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

###Get product names

In [200]:
top_ten_product_names = pd.Series(['Banana',
'Bag of Organic Bananas',
'Organic Strawberries',
'Organic Baby Spinach',
'Organic Hass Avocado',
'Organic Avocado',
'Large Lemon',
'Strawberries',
'Limes',
'Organic Whole Milk']) 

top_ten_product_names

0                    Banana
1    Bag of Organic Bananas
2      Organic Strawberries
3      Organic Baby Spinach
4      Organic Hass Avocado
5           Organic Avocado
6               Large Lemon
7              Strawberries
8                     Limes
9        Organic Whole Milk
dtype: object

How many times was each ordered? 


First, write down which columns you need and which dataframes have them.

I need a field that identifies the item, and a field that identifies an order, filtered to orders that include the items listed.

Item name
Order





### Table: `aisles`

In [201]:
# head_samples = []
aisles = pd.read_csv('aisles.csv')
print(aisles.shape)
# foo = aisles.head
aisles.head()

# print(type(foo))

# foo = aisles.shape
# type(foo)
# # head_samples['aisles'] = aisles.head()
# # head_samples['aisles']
# # loop for fn in ['fn1', 'fn2' ]
# do content = pd.read_csv(fn)
# print(content)
# return ([fn, content.head()])

(134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


**Nothing needed from this table.**

###Table: `departments`

In [202]:
departments = pd.read_csv('departments.csv')
print(departments.shape)
departments.head()

(21, 2)


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


**Nothing needed from this table.**

###Table: `order_products__prior`

In [203]:
order_products__prior = pd.read_csv('order_products__prior.csv')
print(order_products__prior.shape)
order_products__prior.head()

(32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


**I need    `order_id`     and     	`product_id`    from this table.
Also, I need to join this with ..._prior to gather all order data**

###Table: `order_products__train`

In [204]:
order_products__train = pd.read_csv('order_products__train.csv')
print(order_products__train.shape)
order_products__train.head()

(1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


**I need    `order_id`     and     	`product_id`    from this table.
Also, I need to join this with ..._train to gather all order data**

###Table: `orders`

In [205]:
orders = pd.read_csv('orders.csv')
print(orders.shape)
orders.head()

(3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


**Nothing needed from this table.**

###Table: `products`

In [206]:
products = pd.read_csv('products.csv')
print(products.shape)
products.head()

(49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


- products 
 - product name
 - product id
- order_products__all (concatenated table)
 - product id
 - order number

###Get subset of products


In [207]:
# TODO figure out how to make a parsimonious conddition based on a list, 
# set, or series

condition = ((products['product_name'] == 'Banana') 
  | (products['product_name'] == 'Bag of Organic Bananas') 
  | (products['product_name'] == 'Organic Strawberries') 
  | (products['product_name'] == 'Organic Baby Spinach') 
  | (products['product_name'] == 'Organic Hass Avocado') 
  | (products['product_name'] == 'Organic Avocado') 
  | (products['product_name'] == 'Large Lemon') 
  | (products['product_name'] == 'Strawberries') 
  | (products['product_name'] == 'Limes') 
  | (products['product_name'] == 'Organic Whole Milk' ))

products_subset = products.loc[condition, columns]
products_subset

,product_id,product_name
13175,13176,Bag of Organic Bananas
16796,16797,Strawberries
21136,21137,Organic Strawberries
21902,21903,Organic Baby Spinach
24851,24852,Banana
26208,26209,Limes
27844,27845,Organic Whole Milk
47208,47209,Organic Hass Avocado
47625,47626,Large Lemon
47765,47766,Organic Avocado


### Left join between products subset and order_products, joined on product_id

In [0]:
merged = pd.merge(products_subset, 
                  order_products__all, 
                  how='left', 
                  on='product_id')

## Merge order_product tables


Next, merge these into a single dataframe.

In [209]:
# Concat two order_products tables (probably split for ML work)

order_products__all = pd.concat([order_products__prior, order_products__train])

order_products__all.drop(columns=['add_to_cart_order', 'reordered'], inplace=True)

print(order_products__all.shape)
order_products__all.head()

(33819106, 2)


,order_id,product_id
0,2,33120
1,2,28985
2,2,9327
3,2,45918
4,2,30035



## Summarize

### Merge tables with Left join 
between `products_subset` and `order_products__all`, joined on product_id

In [210]:
# Left join between products subset and order_products, joined on product_id
merged = pd.merge(products_subset, 
                  order_products__all, 
                  how='left', 
                  on='product_id')

merged.head(12)

,product_id,product_name,order_id
0,13176,Bag of Organic Bananas,5
1,13176,Bag of Organic Bananas,27
2,13176,Bag of Organic Bananas,29
3,13176,Bag of Organic Bananas,32
4,13176,Bag of Organic Bananas,42
5,13176,Bag of Organic Bananas,55
6,13176,Bag of Organic Bananas,58
7,13176,Bag of Organic Bananas,60
8,13176,Bag of Organic Bananas,67
9,13176,Bag of Organic Bananas,83


### Top 10 by Order Count
Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [211]:
merged['product_name'].value_counts()

Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64

###XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX


## Reshape Data Section

- Replicate the lesson code

In [212]:
from IPython.display import display, Image
url = 'https://cdn-images-1.medium.com/max/1600/1*vYGFQCafJtGBBX5mbl0xyw.png'
example = Image(url=url, width=600)

display(example)

In [213]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


In [214]:
display(example)

In [0]:
import pandas as pd

In [216]:
aisles = pd.read_csv('aisles.csv')
print(aisles.shape)
aisles.head()

(134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [217]:
departments = pd.read_csv('departments.csv')
print(departments.shape)
departments.head()

(21, 2)


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [218]:
order_products__prior = pd.read_csv('order_products__prior.csv')
print(order_products__prior.shape)
order_products__prior.head()

(32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [219]:
order_products__train = pd.read_csv('order_products__train.csv')
print(order_products__train.shape)
order_products__train.head()

(1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [220]:
orders = pd.read_csv('orders.csv')
print(orders.shape)
orders.head()

(3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [221]:
products = pd.read_csv('products.csv')
print(products.shape)
products.head()

(49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [222]:
display(example)

In [223]:
# This is string concatenation
print("Hello " + "World!")

Hello World!


In [224]:
# Concate = stick them together to combine them into one

order_products = pd.concat([order_products__prior, order_products__train])

print(order_products.shape)
order_products.head()

(33819106, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [225]:
display(example)

In [226]:
orders['user_id'] == 1

0           True
1           True
2           True
3           True
4           True
           ...  
3421078    False
3421079    False
3421080    False
3421081    False
3421082    False
Name: user_id, Length: 3421083, dtype: bool

In [227]:
orders['order_number'] <=2

0           True
1           True
2          False
3          False
4          False
           ...  
3421078    False
3421079    False
3421080    False
3421081    False
3421082    False
Name: order_number, Length: 3421083, dtype: bool

In [228]:
# compare lists of true/false values using the bitwise AND operator
# bitwise and: &
# bitwise or: |
condition = (orders['user_id'] == 1) & (orders['order_number'] <=2)

columns = ['user_id', 
           'order_id', 
           'order_number', 
           'order_dow', 
           'order_hour_of_day']

orders_subset = orders.loc[condition, columns]
orders_subset.head()

,user_id,order_id,order_number,order_dow,order_hour_of_day
0,1,2539329,1,2,8
1,1,2398795,2,3,7


In [229]:
condition = ((order_products['order_id'] == 2539329)  
          | (order_products['order_id'] == 2398795))

columns = ['order_id', 'product_id', 'add_to_cart_order']

order_products_subset = order_products.loc[condition, columns]
order_products_subset.head(11)

,order_id,product_id,add_to_cart_order
22742744,2398795,196,1
22742745,2398795,10258,2
22742746,2398795,12427,3
22742747,2398795,13176,4
22742748,2398795,26088,5
22742749,2398795,13032,6
24076664,2539329,196,1
24076665,2539329,14084,2
24076666,2539329,12427,3
24076667,2539329,26088,4


In [230]:
display(example)

In [231]:
# the 'on' parameter indicates a specific column that is contained in *both*
# dataframes, that we will use to lookup and then copy information
# from the two dataframes into a new (singular) combined dataframe

merged = pd.merge(orders_subset, 
                  order_products_subset, 
                  how='inner', 
                  on='order_id')
merged.head(11)

,user_id,order_id,order_number,order_dow,order_hour_of_day,product_id,add_to_cart_order
0,1,2539329,1,2,8,196,1
1,1,2539329,1,2,8,14084,2
2,1,2539329,1,2,8,12427,3
3,1,2539329,1,2,8,26088,4
4,1,2539329,1,2,8,26405,5
5,1,2398795,2,3,7,196,1
6,1,2398795,2,3,7,10258,2
7,1,2398795,2,3,7,12427,3
8,1,2398795,2,3,7,13176,4
9,1,2398795,2,3,7,26088,5


In [232]:
final = pd.merge(merged, 
                 products[['product_id', 'product_name']], 
                 how='left', 
                 on='product_id')
final.head(500)

,user_id,order_id,order_number,order_dow,order_hour_of_day,product_id,add_to_cart_order,product_name
0,1,2539329,1,2,8,196,1,Soda
1,1,2539329,1,2,8,14084,2,Organic Unsweetened Vanilla Almond Milk
2,1,2539329,1,2,8,12427,3,Original Beef Jerky
3,1,2539329,1,2,8,26088,4,Aged White Cheddar Popcorn
4,1,2539329,1,2,8,26405,5,XL Pick-A-Size Paper Towel Rolls
5,1,2398795,2,3,7,196,1,Soda
6,1,2398795,2,3,7,10258,2,Pistachios
7,1,2398795,2,3,7,12427,3,Original Beef Jerky
8,1,2398795,2,3,7,13176,4,Bag of Organic Bananas
9,1,2398795,2,3,7,26088,5,Aged White Cheddar Popcorn


In [233]:
final = final.sort_values(by=['order_number', 'add_to_cart_order'])
final

,user_id,order_id,order_number,order_dow,order_hour_of_day,product_id,add_to_cart_order,product_name
0,1,2539329,1,2,8,196,1,Soda
1,1,2539329,1,2,8,14084,2,Organic Unsweetened Vanilla Almond Milk
2,1,2539329,1,2,8,12427,3,Original Beef Jerky
3,1,2539329,1,2,8,26088,4,Aged White Cheddar Popcorn
4,1,2539329,1,2,8,26405,5,XL Pick-A-Size Paper Towel Rolls
5,1,2398795,2,3,7,196,1,Soda
6,1,2398795,2,3,7,10258,2,Pistachios
7,1,2398795,2,3,7,12427,3,Original Beef Jerky
8,1,2398795,2,3,7,13176,4,Bag of Organic Bananas
9,1,2398795,2,3,7,26088,5,Aged White Cheddar Popcorn


In [234]:
final.columns = [column.replace('_', ' ') for column in final]
final.head(11)

,user id,order id,order number,order dow,order hour of day,product id,add to cart order,product name
0,1,2539329,1,2,8,196,1,Soda
1,1,2539329,1,2,8,14084,2,Organic Unsweetened Vanilla Almond Milk
2,1,2539329,1,2,8,12427,3,Original Beef Jerky
3,1,2539329,1,2,8,26088,4,Aged White Cheddar Popcorn
4,1,2539329,1,2,8,26405,5,XL Pick-A-Size Paper Towel Rolls
5,1,2398795,2,3,7,196,1,Soda
6,1,2398795,2,3,7,10258,2,Pistachios
7,1,2398795,2,3,7,12427,3,Original Beef Jerky
8,1,2398795,2,3,7,13176,4,Bag of Organic Bananas
9,1,2398795,2,3,7,26088,5,Aged White Cheddar Popcorn


In [235]:
display(example)

#ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ

In [0]:
# %matplotlib inline
# import pandas as pd
# import numpy as np
# import seaborn as sns

# table1 = pd.DataFrame(
#     [[np.nan, 2],
#      [16,    11], 
#      [3,      1]],
#     index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
#     columns=['treatmenta', 'treatmentb'])

# table2 = table1.T

# Muscle Memory

%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11],
     [3,      1]],
     index=['John Smith', 'Jane Doe', 'Mary Johnson'],
            columns=['treatmenta', 'treatmentb'])

table2 = table1.T

In [245]:
# table1
table1

,treatmenta,treatmentb
John Smith,NaN,2
Jane Doe,16.0,11
Mary Johnson,3.0,1


In [246]:
#table2
table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [247]:
# Take the row index, and add it as a new column
# table1 = table1.reset_index()
# table1

table1 = table1.reset_index()
table1

,index,treatmenta,treatmentb
0,John Smith,NaN,2
1,Jane Doe,16.0,11
2,Mary Johnson,3.0,1


In [248]:
# What is the unique identifier for each row
# Where is the data at that I want to be in my single "tidy" column
# tidy1 = table1.melt(id_vars='index', value_vars=['treatmenta', 'treatmentb'])
# tidy1

tidy1 = table1.melt(id_vars='index', value_vars=['treatmenta', 'treatmentb'])
tidy1

,index,variable,value
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


In [249]:
# tidy1 = tidy1.rename(columns={
#     'index': 'name', 
#     'variable': 'trt', 
#     'value': 'result'
# })
# tidy1

tidy1 = tidy1.rename(columns={
    'index': 'name',
    'variable': 'trt',
     'value': 'result'
})
tidy1

,name,trt,result
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


In [250]:
# tidy1.trt = tidy1.trt.str.replace('treatment', '')

# tidy1

tidy1.trt=tidy1.trt.str.replace('treatment', '')

tidy1

,name,trt,result
0,John Smith,a,NaN
1,Jane Doe,a,16.0
2,Mary Johnson,a,3.0
3,John Smith,b,2.0
4,Jane Doe,b,11.0
5,Mary Johnson,b,1.0


In [267]:
tidy1.columns

Index(['name', 'trt', 'result'], dtype='object')

###Complete the code cells we skipped near the beginning of the notebook
- Table 2 --> Tidy

In [253]:
table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [255]:
tidy1

,name,trt,result
0,John Smith,a,NaN
1,Jane Doe,a,16.0
2,Mary Johnson,a,3.0
3,John Smith,b,2.0
4,Jane Doe,b,11.0
5,Mary Johnson,b,1.0


In [272]:
# table2 = table2.reset_index()
tidy2 = table2.melt(id_vars='index', value_vars=['John Smith', 'Jane Doe', 'Mary Johnson'])
tidy2

,index,variable,value
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


- Tidy --> Table 2

In [273]:
table2

,index,John Smith,Jane Doe,Mary Johnson
0,treatmenta,NaN,16.0,3.0
1,treatmentb,2.0,11.0,1.0


In [275]:
tidy2

,index,variable,value
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [278]:
tidy2.columns

Index(['index', 'variable', 'value'], dtype='object')

In [283]:
# I have tried each of the following, no joy.

# wide2 = tidy2.pivot_table(index=['treatmenta','treatmentb'], columns='name', values='result')
# wide2 = tidy2.pivot_table(index='treatment', columns='name', values='result')
# wide2 = tidy2.pivot_table(index='trt', columns='name', values='result')
wide2 = tidy2.pivot_table(index='index', columns='name', values='result')
wide

KeyError: ignored


- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [0]:
flights = sns.load_dataset('flights')

In [0]:
##### YOUR CODE HERE #####

## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [0]:
##### YOUR CODE HERE #####

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####